#### This notebook reads ``us_addresses.json`` and calls ``api.openalpr.com`` to get car metdata like model, colour, type etc. The output file it creates is a JSON file. 

In [4]:
#!/usr/bin/python

import requests
import base64
import json
import uuid
#pip3 install Faker
from faker import Faker
import random
from geopy.geocoders import Nominatim

def seed_db_creator(IMAGE_PATH):
  SECRET_KEY = 'sk_90c281aebbfc04d52d6c2e860KR'
  results= {}
  with open(IMAGE_PATH, 'rb') as image_file:
      img_base64 = base64.b64encode(image_file.read())
  url = 'https://api.openalpr.com/v3/recognize_bytes?recognize_vehicle=1&country=us&secret_key=%s' % (SECRET_KEY)
  r = requests.post(url, data = img_base64)
  data = json.loads( json.dumps(r.json(), indent=2))
  
  ## Uncomment if you want to get input data from json instead of openalpr.com API
  dump_file = open("dump.json")
  data = json.load(dump_file)

  address_file = open("us_addresses.json")
  address_data = json.load(address_file)
  
  for i in data["results"]:
    customer_id = str(uuid.uuid4())
    fake_owner_name = fake.name()
    fake_owner_contact_number = fake.numerify('+1-###-###-####')
    fake_address_details = fake.address()
    fake_address_details = address_data["addresses"][random.randint(0,999)]
    #print(fake_address_details)
    fake_lat_long = fake.local_latlng()

    ##### Vehicle Information #####
    results.update({"vehicle_registered_plate_number": i['plate']})
    results.update({"vehicle_color": i["vehicle"]["color"][0]["name"]})
    results.update({"vehicle_make": i["vehicle"]["make"][0]["name"]})
    results.update({"vehicle_body_type": i["vehicle"]["body_type"][0]["name"]})
    results.update({"vehicle_make_model": i["vehicle"]["make_model"][0]["name"]})
    results.update({"vehicle_model_year": i["vehicle"]["year"][0]["name"]})
    results.update({"vehicle_registered_locality": fake_address_details['state']})
   ###### Owner Information #####
    results.update({"vehicle_owner_name": fake_owner_name})
    results.update({"vehicle_owner_address": fake_address_details['address1']})
    results.update({"vehicle_owner_city": fake_address_details['city']})
    results.update({"vehicle_owner_state": fake_address_details['state']})
    results.update({"vehicle_owner_zip_code": fake_address_details['postalCode']})
    results.update({"vehicle_owner_contact_number": fake_owner_contact_number})
   ###### Customer Information #####
    results.update({"customer_id": customer_id})
    results.update({"customer_balance": str(random.randint(100,999)) + " USD"})
    results.update({"customer_name": fake_owner_name})
    results.update({"customer_address": fake_address_details['address1']})
    results.update({"customer_city": fake_address_details['city']})
    results.update({"customer_state": fake_address_details['state']})
    results.update({"customer_zip_code": fake_address_details['postalCode']})
    results.update({"customer_locality": fake_address_details['state']})
    results.update({"customer_contact_number": fake_owner_contact_number})
    temp = final_results['seed_database']
    temp.append(results)
    with open('database.json', 'w', encoding='utf-8') as f:
      json.dump(temp, f, ensure_ascii=False, indent=4)
    print(temp)  



final_results = {"seed_database": [ ] }
#fake = Faker(['en-GB', 'en_GB', 'en_GB', 'en-GB'])
fake = Faker( 'en_GB')
#print(fake.locales)

for i in range(11,12):
  IMAGE_PATH = '/Users/karasing/git/jumpstart-library/pattern2-licence-plates/LPR_Service/dataset/images/'+str(i)+'.png'
  #print(IMAGE_PATH)
  seed_db_creator(IMAGE_PATH)


[{'vehicle_registered_plate_number': 'CHIAN000', 'vehicle_color': 'black', 'vehicle_make': 'toyota', 'vehicle_body_type': 'sedan-wagon', 'vehicle_make_model': 'toyota_land-cruiser', 'vehicle_model_year': '2010-2014', 'vehicle_registered_locality': 'MA', 'vehicle_owner_name': 'Roy Dale-Robinson', 'vehicle_owner_address': '302 Depot Road', 'vehicle_owner_city': 'Boxborough', 'vehicle_owner_state': 'MA', 'vehicle_owner_zip_code': '01719', 'vehicle_owner_contact_number': '+1-873-399-6088', 'customer_id': 'afc220a0-ccee-414e-99df-73300b555d53', 'customer_balance': '164 USD', 'customer_name': 'Roy Dale-Robinson', 'customer_address': '302 Depot Road', 'customer_city': 'Boxborough', 'customer_state': 'MA', 'customer_zip_code': '01719', 'customer_locality': 'MA', 'customer_contact_number': '+1-873-399-6088'}]


In [ ]:

## For DB
# vehicle_registered_plate_number
# vehicle_color
# vehicle_make
# vehicle_body_type
# vehicle_make_model
# vehicle_model_year
# vehicle_registered_locality

# vehicle_owner_name
# vehicle_owner_address
# vehicle_owner_contact_number

# customer_id
# customer_balance
# customer_name
# customer_address
# customer_locality
# customer_contact_number

## To Kafka
# event_id
# event_vehicle_detected_plate_number
# event_vehicle_captured_image
# event_vehicle_detected_geo_location
# event_timestamp